In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [27]:
!pip install kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mishra5001/credit-card")

print("Path to dataset files:", path)

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Path to dataset files: /Users/shehab/.cache/kagglehub/datasets/mishra5001/credit-card/versions/1


In [28]:
app_df = pd.read_csv(path+"/application_data.csv")
app_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
app_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


In [30]:
prev_app_df = pd.read_csv(path+"/previous_application.csv")
prev_app_df.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
merged_df = app_df.merge(prev_app_df, on='SK_ID_CURR', how='inner')
for col in merged_df.columns:
    if merged_df[col].dtype in ['float64', 'object']:
        if merged_df[col].dropna().apply(lambda x: str(x).replace('.', '', 1).isdigit()).all():
            merged_df[col] = merged_df[col].astype(int, errors='ignore')
merged_df.info()
merged_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1413701 entries, 0 to 1413700
Columns: 158 entries, SK_ID_CURR to NFLAG_INSURED_ON_APPROVAL
dtypes: float64(75), int64(51), object(32)
memory usage: 1.7+ GB


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,100002,1,Cash loans,M,N,Y,0,202500,406597,24700.5,...,Auto technology,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0
1,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0
2,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,Furniture,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0
3,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,Consumer electronics,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0
4,100004,0,Revolving loans,M,Y,Y,0,67500,135000,6750.0,...,Connectivity,4.0,middle,POS mobile without interest,365243.0,-784.0,-694.0,-724.0,-714.0,0.0


In [32]:
merged_df.dtypes

SK_ID_CURR                     int64
TARGET                         int64
NAME_CONTRACT_TYPE_x          object
CODE_GENDER                   object
FLAG_OWN_CAR                  object
                              ...   
DAYS_FIRST_DUE               float64
DAYS_LAST_DUE_1ST_VERSION    float64
DAYS_LAST_DUE                float64
DAYS_TERMINATION             float64
NFLAG_INSURED_ON_APPROVAL    float64
Length: 158, dtype: object

In [33]:
transactions = merged_df[['SK_ID_CURR', 'SK_ID_PREV']].copy()
transactions = transactions.rename(columns={'SK_ID_PREV': 'SK_ID_RECEIVER'})

In [34]:
class UnionFind:
    def __init__(self, elements):
        self.parent = {element: element for element in elements}
        self.rank = {element: 0 for element in elements}

    def find(self, element):
        if self.parent[element] != element:
            self.parent[element] = self.find(self.parent[element]) # Path compression
        return self.parent[element]

    def union(self, element1, element2):
        root1 = self.find(element1)
        root2 = self.find(element2)

        if root1 != root2:
            if self.rank[root1] < self.rank[root2]:
                self.parent[root1] = root2
            elif self.rank[root1] > self.rank[root2]:
                self.parent[root2] = root1
            else:
                self.parent[root2] = root1
                self.rank[root1] += 1

In [35]:
unique_accounts = pd.concat([transactions['SK_ID_CURR'], transactions['SK_ID_RECEIVER']]).unique()
uf = UnionFind(unique_accounts)

for index, row in transactions.iterrows():
    uf.union(row['SK_ID_CURR'], row['SK_ID_RECEIVER'])

In [36]:
account_community_map = {account: uf.find(account) for account in unique_accounts}
merged_df['community_id'] = merged_df['SK_ID_CURR'].map(account_community_map)

In [37]:
merged_df.to_csv('merged_credit_card_data.csv', index=False)
print("merged_credit_card_data.csv has been saved.")

merged_credit_card_data.csv has been saved.


In [38]:
merged_df.T

,0,1,2,3,4,5,6,7,8,9,...,1413691,1413692,1413693,1413694,1413695,1413696,1413697,1413698,1413699,1413700
SK_ID_CURR,100002,100003,100003,100003,100004,100006,100006,100006,100006,100006,...,456254,456254,456255,456255,456255,456255,456255,456255,456255,456255
TARGET,1,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
NAME_CONTRACT_TYPE_x,Cash loans,Cash loans,Cash loans,Cash loans,Revolving loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,...,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans
CODE_GENDER,M,F,F,F,M,F,F,F,F,F,...,F,F,F,F,F,F,F,F,F,F
FLAG_OWN_CAR,N,N,N,N,Y,N,N,N,N,N,...,N,N,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DAYS_LAST_DUE_1ST_VERSION,125.0,-386.0,-647.0,-1980.0,-694.0,1259.0,NaN,49.0,365243.0,NaN,...,203.0,99.0,-630.0,-95.0,28.0,72.0,NaN,NaN,294.0,-319.0
DAYS_LAST_DUE,-25.0,-536.0,-647.0,-1980.0,-724.0,-151.0,NaN,365243.0,365243.0,NaN,...,365243.0,365243.0,-690.0,-215.0,-212.0,-558.0,NaN,NaN,-66.0,-409.0
DAYS_TERMINATION,-17.0,-527.0,-639.0,-1976.0,-714.0,-143.0,NaN,365243.0,365243.0,NaN,...,365243.0,365243.0,-687.0,-211.0,-204.0,-528.0,NaN,NaN,-64.0,-400.0
NFLAG_INSURED_ON_APPROVAL,0.0,1.0,0.0,1.0,0.0,0.0,NaN,0.0,0.0,NaN,...,1.0,0.0,0.0,0.0,1.0,1.0,NaN,NaN,0.0,0.0


In [39]:
merged_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE_x,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY_x,...,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,community_id
0,100002,1,Cash loans,M,N,Y,0,202500,406597,24700.5,...,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0,100002
1,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0,100003
2,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0,100003
3,100003,0,Cash loans,F,N,N,0,270000,1293502,35698.5,...,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0,100003
4,100004,0,Revolving loans,M,Y,Y,0,67500,135000,6750.0,...,4.0,middle,POS mobile without interest,365243.0,-784.0,-694.0,-724.0,-714.0,0.0,100004


In [40]:
merged_df.isnull().sum()

SK_ID_CURR                        0
TARGET                            0
NAME_CONTRACT_TYPE_x              0
CODE_GENDER                       0
FLAG_OWN_CAR                      0
                              ...  
DAYS_LAST_DUE_1ST_VERSION    561106
DAYS_LAST_DUE                561106
DAYS_TERMINATION             561106
NFLAG_INSURED_ON_APPROVAL    561106
community_id                      0
Length: 159, dtype: int64

In [41]:
null_summary = (
    merged_df.isnull()
    .sum()
    .reset_index()
    .rename(columns={'index': 'Feature', 0: 'Null_Count'})
)

null_summary['Null_%'] = (null_summary['Null_Count'] / len(merged_df) * 100).round(2)
null_summary = null_summary.sort_values(by='Null_Count', ascending=False)

print(null_summary)

                         Feature  Null_Count  Null_%
134        RATE_INTEREST_PRIMARY     1408910   99.66
135     RATE_INTEREST_PRIVILEGED     1408910   99.66
48                COMMONAREA_AVG      985199   69.69
76               COMMONAREA_MEDI      985199   69.69
62               COMMONAREA_MODE      985199   69.69
..                           ...         ...     ...
36   LIVE_REGION_NOT_WORK_REGION           0    0.00
35    REG_REGION_NOT_WORK_REGION           0    0.00
34    REG_REGION_NOT_LIVE_REGION           0    0.00
33     HOUR_APPR_PROCESS_START_x           0    0.00
158                 community_id           0    0.00

[159 rows x 3 columns]


In [42]:
def drop_high_null_features(merged_df, threshold=0.4):
    """
    Drops all columns with more than 'threshold' proportion of null values.
    
    Parameters:
        df (pd.DataFrame): Input DataFrame
        threshold (float): Fraction of null values above which columns are dropped (default = 0.4)
        
    Returns:
        pd.DataFrame: Cleaned DataFrame with columns removed
    """
    # Calculate the fraction of nulls per column
    null_fraction = merged_df.isnull().mean()
    
    # Columns to drop
    cols_to_drop = null_fraction[null_fraction > threshold].index
    
    print(f"🧹 Dropping {len(cols_to_drop)} columns with more than {threshold*100}% null values")
    
    # Drop and return
    df_cleaned = merged_df.drop(columns=cols_to_drop)
    return df_cleaned

In [ ]:
merged_df_clean = drop_high_null_features(merged_df, threshold=0.4)
merged_df_clean.shape

🧹 Dropping 54 columns with more than 40.0% null values


(1413701, 105)